목표 : 8/7~11/3 strike point
---------------

import modules

In [140]:
import pandas as pd
import numpy as np
import math
import glob
import datetime
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1


load file

In [141]:
ks200 = pd.read_csv('./datatset/data/KOSPI200.csv', index_col=0)

In [142]:
ks200.info()

<class 'pandas.core.frame.DataFrame'>
Index: 260 entries, 2023- 10- 31 to 2022- 10- 13
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종가      260 non-null    float64
 1   시가      260 non-null    float64
 2   고가      260 non-null    float64
 3   저가      260 non-null    float64
 4   거래량     260 non-null    object 
 5   변동 %    260 non-null    object 
dtypes: float64(4), object(2)
memory usage: 14.2+ KB


In [143]:
# 음수 값이 있을수 있으므로 절댓값 취하기
ks200 = ks200['종가'].abs()

In [144]:
ks200.index = pd.to_datetime(ks200.index, format="%Y- %m- %d")

In [145]:
# 날짜 7월 31일부터 -> 옵션은 전날 종가로 기준 행사가를 잡음
# ks200 = ks200.loc[ks200['일자'] > 20230727]

In [146]:
ks200.head(3)

날짜
2023-10-31    306.33
2023-10-30    308.95
2023-10-27    308.53
Name: 종가, dtype: float64

##### make dataframe strike price

In [147]:
df = ks200.copy()

In [148]:
df

날짜
2023-10-31    306.33
2023-10-30    308.95
2023-10-27    308.53
2023-10-26    307.75
2023-10-25    316.49
               ...  
2022-10-19    291.29
2022-10-18    293.59
2022-10-17    289.57
2022-10-14    289.36
2022-10-13    282.57
Name: 종가, Length: 260, dtype: float64

In [149]:
# 소수점만 가져오기
result = df.apply(lambda x: x % 10)

In [150]:
result

날짜
2023-10-31    6.33
2023-10-30    8.95
2023-10-27    8.53
2023-10-26    7.75
2023-10-25    6.49
              ... 
2022-10-19    1.29
2022-10-18    3.59
2022-10-17    9.57
2022-10-14    9.36
2022-10-13    2.57
Name: 종가, Length: 260, dtype: float64

In [151]:
result = pd.DataFrame(result)
result.columns = ['소수']

In [152]:
result

,소수
날짜,
2023-10-31,6.33
2023-10-30,8.95
2023-10-27,8.53
2023-10-26,7.75
2023-10-25,6.49
...,...
2022-10-19,1.29
2022-10-18,3.59
2022-10-17,9.57


In [153]:
price = pd.concat([df, result], axis=1)

In [154]:
price.head(3)

,종가,소수
날짜,,
2023-10-31,306.33,6.33
2023-10-30,308.95,8.95
2023-10-27,308.53,8.53


In [155]:
# 행사가 컬럼 생성
price['행사가'] = 0

In [156]:
# 소수에 따라 기준 행사가 지정
for index, i in enumerate(price['소수']):
	if i > 7.5:
		price['행사가'][index] = 7.5
	elif i > 5:
		price['행사가'][index] = 5
	elif i > 2.5:
		price['행사가'][index] = 2.5
	else:
		price['행사가'][index] = 0

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_13824\2772650412.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['행사가'][index] = 5
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_13824\2772650412.py:6: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  price['행사가'][index] = 5
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_13824\2772650412.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['행사가'][index] = 7.5
C:\Users\ja

In [157]:
price

,종가,소수,행사가
날짜,,,
2023-10-31,306.33,6.33,5.0
2023-10-30,308.95,8.95,7.5
2023-10-27,308.53,8.53,7.5
2023-10-26,307.75,7.75,7.5
2023-10-25,316.49,6.49,5.0
...,...,...,...
2022-10-19,291.29,1.29,0.0
2022-10-18,293.59,3.59,2.5
2022-10-17,289.57,9.57,7.5


In [158]:
price['기준행사가'] = price['행사가'] + (price['종가'].apply(lambda x: x // 10) * 10)

In [159]:
price

,종가,소수,행사가,기준행사가
날짜,,,,
2023-10-31,306.33,6.33,5.0,305.0
2023-10-30,308.95,8.95,7.5,307.5
2023-10-27,308.53,8.53,7.5,307.5
2023-10-26,307.75,7.75,7.5,307.5
2023-10-25,316.49,6.49,5.0,315.0
...,...,...,...,...
2022-10-19,291.29,1.29,0.0,290.0
2022-10-18,293.59,3.59,2.5,292.5
2022-10-17,289.57,9.57,7.5,287.5


### 기준행사가

In [160]:
strike_price = price[['기준행사가']]

기준행사가 및 (+-4)

In [161]:
strike_price['S-4'] = strike_price['기준행사가'] - 10
strike_price['S-3'] = strike_price['기준행사가'] - 7.5
strike_price['S-2'] = strike_price['기준행사가'] - 5
strike_price['S-1'] = strike_price['기준행사가'] - 2.5
strike_price['S+1'] = strike_price['기준행사가'] + 2.5
strike_price['S+2'] = strike_price['기준행사가'] + 5
strike_price['S+3'] = strike_price['기준행사가'] + 7.5
strike_price['S+4'] = strike_price['기준행사가'] + 10

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_13824\1700255692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strike_price['S-4'] = strike_price['기준행사가'] - 10
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_13824\1700255692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strike_price['S-3'] = strike_price['기준행사가'] - 7.5


In [162]:
strike_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 260 entries, 2023-10-31 to 2022-10-13
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준행사가   260 non-null    float64
 1   S-4     260 non-null    float64
 2   S-3     260 non-null    float64
 3   S-2     260 non-null    float64
 4   S-1     260 non-null    float64
 5   S+1     260 non-null    float64
 6   S+2     260 non-null    float64
 7   S+3     260 non-null    float64
 8   S+4     260 non-null    float64
dtypes: float64(9)
memory usage: 20.3 KB


In [163]:
strike_price

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,305.0,295.0,297.5,300.0,302.5,307.5,310.0,312.5,315.0
2023-10-30,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-27,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-26,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-25,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
...,...,...,...,...,...,...,...,...,...
2022-10-19,290.0,280.0,282.5,285.0,287.5,292.5,295.0,297.5,300.0
2022-10-18,292.5,282.5,285.0,287.5,290.0,295.0,297.5,300.0,302.5
2022-10-17,287.5,277.5,280.0,282.5,285.0,290.0,292.5,295.0,297.5


In [164]:
# strike_price = strike_price.set_index('일자')

In [165]:
# 전날 종가가 당일 기준행사가이므로 행 밀어주기
strike_price = strike_price.shift(-1)

In [166]:
strike_price.head(22)

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-30,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-27,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-26,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-10-25,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-10-24,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-10-23,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-10-20,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
2023-10-19,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5


In [167]:
# strike_price.drop(index=[20230728, 20231023, 20231024], axis=0, inplace=True)

In [168]:
strike_price.index

DatetimeIndex(['2023-10-31', '2023-10-30', '2023-10-27', '2023-10-26',
               '2023-10-25', '2023-10-24', '2023-10-23', '2023-10-20',
               '2023-10-19', '2023-10-18',
               ...
               '2022-10-26', '2022-10-25', '2022-10-24', '2022-10-21',
               '2022-10-20', '2022-10-19', '2022-10-18', '2022-10-17',
               '2022-10-14', '2022-10-13'],
              dtype='datetime64[ns]', name='날짜', length=260, freq=None)

# 기간조정

In [169]:
strike_price.index

DatetimeIndex(['2023-10-31', '2023-10-30', '2023-10-27', '2023-10-26',
               '2023-10-25', '2023-10-24', '2023-10-23', '2023-10-20',
               '2023-10-19', '2023-10-18',
               ...
               '2022-10-26', '2022-10-25', '2022-10-24', '2022-10-21',
               '2022-10-20', '2022-10-19', '2022-10-18', '2022-10-17',
               '2022-10-14', '2022-10-13'],
              dtype='datetime64[ns]', name='날짜', length=260, freq=None)

In [170]:
start_date = pd.to_datetime("2023-08-07")
end_date = pd.to_datetime("2023-11-03")
strike_price = strike_price.loc[end_date:start_date]

# to_csv

In [171]:
# strike_price.to_csv('./dataset/strike_price.csv')

In [172]:
strike_price

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-30,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-27,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-26,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-10-25,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-10-24,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-10-23,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-10-20,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
2023-10-19,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5


In [173]:
# oct = strike_price[:9]
# sep = strike_price[9:25]
# aug = strike_price[25:44]
# july = strike_price[44:]

In [174]:
# 해당 달 별 최소 행사가와 최대 행사가 찾기
# oct.describe()